## Import lib.

In [2]:
import os
import zipfile


import cv2
import matplotlib.pyplot as plt 
import numpy as np 
import torch 
import torch.nn as nn 

In [17]:
with open(".env") as f:
    for line in f:
        key, val = line.strip().split("=")
        os.environ[key] = val

# print("Kaggle user:", os.environ['KAGGLE_USERNAME'])  
# print("Kaggle user:", os.environ['KAGGLE_KEY'])  